<a href="https://colab.research.google.com/github/AR6420/TwitterSentimentAnalysis/blob/main/Twitter_SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option('display.max_colwidth', None)
pd.set_option("display.max_rows", None)

In [ ]:
df = pd.read_csv('Ukraine_tweetys.txt')

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.head()

In [ ]:
# Find tweets in English only
df_en = df[df['language']=='en']

In [ ]:
df_en.shape

In [ ]:
# Remove unnecessary columns
df_en_clean = df_en.drop(columns = ['Tweet Id','Unnamed: 0','Unnamed: 0.1','verified'], axis  = 1)

In [ ]:
df_en_clean.head()

In [ ]:
df_en_clean['Username'].value_counts()

In [ ]:
df_en_clean['location'].value_counts()

In [ ]:
df_en_clean['like count'].value_counts()

In [ ]:
df_en_clean[df_en_clean['Username']=='StopVladdyDaddy']['Follower Count'].value_counts()

In [ ]:
df_en_clean['Text'].head(10)

In [ ]:
df_en_clean.to_csv('cleaned_tweets.csv')

## Tweets from Russia location

In [ ]:
import numpy as np
position = 0
positions = []
for location in df_en_clean.location:
    if type(location) != float:
        location = location.lower().split()
        if 'russia' in location and 'us' not in location and 'not' not in location and 'but' not in location:
            positions.append(position)
    else:
        pass
    position = position + 1

In [ ]:
df_en_clean.iloc[positions[1],3]

In [ ]:
df_russia_en = pd.DataFrame()
for position in positions:
    entry = df_en_clean.iloc[position,:]
    #print(row)
    df_russia_en= pd.concat([df_russia_en,entry],axis = 1)
    #row_2 = df.loc[1].copy()

In [ ]:
df_russia_en = df_russia_en.transpose()

In [ ]:
df_russia_en.location.value_counts()

In [ ]:
df_russia_en.shape

In [ ]:
df_russia_en.to_csv('Russia_loc.csv')

## Tweets from Ukraine location

In [ ]:
import numpy as np
position = 0
positions = []
for location in df_en_clean.location:
    if type(location) != float:
        location = location.lower().split()
        if ('ukraine' in location and 'taxpayer' not in location and\
            'and' not in location and 'in' not in location\
            and 'us' not in location and 'stand' not in location\
            and 'is' not in location and 'free' not in location\
           and 'earth' not in location and 'uk' not in location\
            and 'with' not in location and 'germany' not in location\
           and 'ireland' not in location and 'to' not in location\
           and 'support' not in location and 'ca' not in location\
           and 'ny' not in location and 'all' not in location\
           and 'or' not in location and 'denmark' not in location):
            positions.append(position)
    else:
        pass
    position = position + 1
print(df_en_clean.iloc[positions[0],:])

In [ ]:
df_ukraine_en = pd.DataFrame()
for position in positions:
    entry = df_en_clean.iloc[position,:]
    #print(entry.location)
    df_ukraine_en = pd.concat([df_ukraine_en,entry],axis = 1)

In [ ]:
df_ukraine_en = df_ukraine_en.transpose()

In [ ]:
df_ukraine_en.shape

In [ ]:
df_ukraine_en.head()

In [ ]:
df_ukraine_en.location.value_counts()

In [ ]:
df_ukraine_en.to_csv('Ukraine_loc.csv')

## Tweets from all other location

In [ ]:
df_other_en = pd.concat([df_en_clean, df_ukraine_en, df_russia_en]).drop_duplicates(keep=False)

In [ ]:
df_other_en.shape

In [ ]:
df_other_en.to_csv('Other_loc.csv')

## High impact tweets (like, retweet and reply > 1)

In [ ]:
df_en_highimpact = df_en_clean[df_en_clean['like count'] > 1]
df_en_highimpact = df_en_highimpact[df_en_highimpact['retweet count']>1]
df_en_highimpact = df_en_highimpact[df_en_highimpact['reply count']>1]
df_en_highimpact.shape

In [ ]:
df_en_highimpact.to_csv('highimpact.csv')

## Topic modeling for high impact tweets

In [ ]:
import gensim
from gensim.utils import simple_preprocess
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import spacy
import gensim.corpora as corpora

In [ ]:
# clean_text takes in a pd.Series and removes special characters, stopwords, etc
def clean_text_nltk(pd_column):
    stop_words = list(set(stopwords.words('english')))  #use nltk stopwords
    clean = []
    for i in pd_column:
        i = i.lower()  #turn to lower case
        #print(i)

        #preprocess to split the words
        #re.split() has problem with '-' if no space between words, so first do i.split('-')
        i = i.split('-')
        i = ' '.join(i)
        i = i.split()
        #print(i)

        split_sent2 = []

        #remove stopwords and special characters
        for word in i:

            new_word = []
            if word not in stop_words and word[0]!='@' and word[0:4] != 'http' and word[0]!='#':
            #check each word to remove stop words, @username, website, hash-tag

                for letter in word:  #check each letter in word to remove special characters
                    if letter.isalpha(): #or letter.isdigit():
                        new_word.append(letter)
                    else:
                        pass
                word2 = ''.join(new_word)
                #print(word2)

                if word2 in stop_words or len(word2) < 3: #check stopwords again
                    pass
                else:
                    split_sent2.append(word2)
            else:
                pass

        #print(split_sent2)

        sent2 = ' '.join(split_sent2)
        clean.append(sent2)

    #print(clean[0:10])
    return(clean)

In [ ]:
test = clean_text_nltk(df_en_highimpact['Text'])

In [ ]:
test[0:5]

In [ ]:
# Tokenization, input is pd.Series with full sentences
from nltk.tokenize import sent_tokenize, word_tokenize

def nltk_tokenizer(pd_column):
    tokenized = []
    for i in pd_column:
        word_tokens = word_tokenize(i)
        tokenized.append(word_tokens)
    return tokenized

In [ ]:
df_tokenized = pd.Series(nltk_tokenizer(test))
df_tokenized.head()

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(df_tokenized, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[df_tokenized], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
# See trigram example
print(trigram_mod[bigram_mod[df_tokenized[0]]])

In [ ]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [ ]:
data_words_bigrams = make_bigrams(df_tokenized)
print(data_words_bigrams[0:5])

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
#print(data_lemmatized)

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5,
                                           random_state=100)

In [ ]:
from pprint import pprint
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
lda_model2 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=100)

In [ ]:
pprint(lda_model2.print_topics())
doc_lda2 = lda_model2[corpus]

In [ ]:
lda_model3 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=7,
                                           random_state=100)

In [ ]:
pprint(lda_model3.print_topics())
doc_lda2 = lda_model2[corpus]

In [ ]:
pip install pyLDAvis

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model2, corpus, id2word)
vis

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model3, corpus, id2word)
vis

In [ ]:
lemmatized_tweets = []
for ls in data_lemmatized:
    tweet = ' '.join(ls)
    lemmatized_tweets.append(tweet)

lemmatized_tweets[0:5]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
dtm = tfidf.fit_transform(lemmatized_tweets)

In [ ]:
from sklearn.decomposition import NMF
nmf_model = NMF(n_components=5,random_state=42) #choosing 5 topics
nmf_model.fit(dtm)
tfidf.get_feature_names_out()[2400]

In [ ]:
topic_lists = []
for index,topic in enumerate(nmf_model.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    topic_list=[tfidf.get_feature_names_out()[i] for i in topic.argsort()[-30:]]
    print([tfidf.get_feature_names_out()[i] for i in topic.argsort()[-30:]])
    topic_lists.append(topic_list)
    print('\n')

In [ ]:
topic_list1 = topic_lists[0]
topic_list2 = topic_lists[1]
topic_list3 = topic_lists[2]
topic_list4 = topic_lists[3]
topic_list5 = topic_lists[4]

In [ ]:
set(topic_list1)&set(topic_list2)&set(topic_list3)&set(topic_list4)&set(topic_list5)

In [ ]:
topic_results = nmf_model.transform(dtm)
df_en_highimpact['Topic'] = topic_results.argmax(axis=1)

In [ ]:
df_en_highimpact.head()

In [ ]:
mytopic_dict = {0:'War cause',1:'Millitary progress',2:'Refugee and human rights',
                3:'Economic impact of the war',4:'News and journalism in war'}
df_en_highimpact['Topic Label'] = df_en_highimpact['Topic'].map(mytopic_dict)

In [ ]:
df_en_highimpact.head()

In [ ]:
df_en_highimpact['Topic'].value_counts()

In [ ]:
df_en_highimpact.to_csv('highimpact_nmf.csv')

## Sentiment analysis for high impact tweets

In [ ]:
# Import SentimentIntensityAnalyzer and create an sid object
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()

In [ ]:
def sentiment_rating(string):
    sid_dict = sid.polarity_scores(string)
    if sid_dict['compound']>= 0.33:
        return ('Positive')
    elif sid_dict['compound']<= -0.33:
        return ('Negative')
    elif sid_dict['compound'] > -0.33 and sid_dict['compound'] < 0.33:
        return('Neutral')
    else:
        return('Score out of range')

sentiment_nltk = []
for tweet in df_en_highimpact['Text']:
    sid_dict = sentiment_rating(tweet)
    sentiment_nltk.append(sid_dict)

df_en_highimpact['sentiment_nltk'] = sentiment_nltk
#df_en_highimpact['sentiment_rating'] = sentimen

In [ ]:
df_en_highimpact['sentiment_nltk'].value_counts()

## Sentiment analysis for Russia location tweets

In [ ]:
def sentiment_rating(string):
    sid_dict = sid.polarity_scores(string)
    if sid_dict['compound']>= 0.33:
        return ('Positive')
    elif sid_dict['compound']<= -0.33:
        return ('Negative')
    elif sid_dict['compound'] > -0.33 and sid_dict['compound'] < 0.33:
        return('Neutral')
    else:
        return('Score out of range')

sentiment_nltk = []
for tweet in df_russia_en['Text']:
    sid_dict = sentiment_rating(tweet)
    sentiment_nltk.append(sid_dict)


df_russia_en['sentiment_nltk'] = sentiment_nltk

In [ ]:
df_russia_en.sentiment_nltk.value_counts()

In [ ]:
sentiment_nltk = []
for tweet in df_russia_en['Text']:
    sid_dict = sentiment_rating(tweet)
    sentiment_nltk.append(sid_dict)

df_russia_en['sentiment_rating'] = sentiment_nltk

In [ ]:
df_russia_en.sentiment_rating.value_counts()

In [ ]:
df_russia_en[df_russia_en['sentiment_nltk'] == 'Positive'].head()

In [ ]:
df_russia_en[df_russia_en['sentiment_nltk'] == 'Neutral'].head()

In [ ]:
df_russia_en[df_russia_en['sentiment_nltk'] == 'Negative'].head()

## Sentiment analysis for Ukraine location tweets

In [ ]:
def sentiment_rating(string):
    sid_dict = sid.polarity_scores(string)
    if sid_dict['compound']>= 0.33:
        return ('Positive')
    elif sid_dict['compound']<= -0.33:
        return ('Negative')
    elif sid_dict['compound'] > -0.33 and sid_dict['compound'] < 0.33:
        return('Neutral')
    else:
        return('Score out of range')

sentiment_nltk = []
for tweet in df_ukraine_en['Text']:
    sid_dict = sentiment_rating(tweet)
    sentiment_nltk.append(sid_dict)


df_ukraine_en['sentiment_nltk'] = sentiment_nltk

In [ ]:
df_ukraine_en.sentiment_nltk.value_counts()

In [ ]:
df_ukraine_en[df_ukraine_en['sentiment_nltk'] == 'Positive'].head()

In [ ]:
df_ukraine_en[df_ukraine_en['sentiment_nltk'] == 'Neutral'].head()

In [ ]:
df_ukraine_en[df_ukraine_en['sentiment_nltk'] == 'Negative'].head()